# CL Test

## Installation
Install all the necessary library.

In [ ]:
! pip install avalanche-lib==0.3.1
! pip install micromind

In [5]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam

import numpy as np
import matplotlib.pyplot as plt

from avalanche.benchmarks.classic import SplitMNIST
from avalanche.models import SimpleMLP
from avalanche.training.storage_policy import ReservoirSamplingBuffer


from strategy.joint_training import JointTraining
from strategy.fine_tuning import FineTuning
from strategy.comulative import Comulative
from strategy.replay import Replay
from strategy.latent_replay import LatentReplay
from phinet_v2 import PhiNet_v2
from utils import get_MAC

from micromind import PhiNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

torch.manual_seed(0)

Device: cpu


## Benchmark

In [ ]:
split_mnist = SplitMNIST(n_experiences=5, seed=0, return_task_id = True)

# recovering the train and test streams
train_stream = split_mnist.train_stream
test_stream = split_mnist.test_stream

## Training

### Training Parameters

In [ ]:
criterion = nn.CrossEntropyLoss()

# Define other necessary parameters
train_mb_size = 128
eval_mb_size = 128

### Fine Tuning Strategy

In [ ]:
model1 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=7 ,include_top = True, num_classes = 10).to(device)

optimizer1 = Adam(model1.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

fine_tuning = FineTuning(
    model=model1,
    optimizer=optimizer1,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

fine_tuning.train(train_stream)
b,c = fine_tuning.test(test_stream)
a = fine_tuning.get_tasks_acc()
print(f"a: {a}, b: {b}, c: {c}")


### Joint Training Strategy

In [ ]:
model2 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=7 ,include_top = True, num_classes = 10).to(device)

optimizer2 = Adam(model2.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

joint_training = JointTraining(
    model=model2,
    optimizer=optimizer2,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

joint_training.train(train_stream)
b,c = joint_training.test(test_stream)
a = joint_training.get_tasks_acc()
print(f"a: {a}, b: {b}, c: {c}")

### Comulative Strategy

In [ ]:
model3 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=7 ,include_top = True, num_classes = 10).to(device)

optimizer3 = Adam(model3.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

comulative = Comulative(
    model=model3,
    optimizer=optimizer3,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

comulative.train(train_stream)
b,c = comulative.test(test_stream)
a = comulative.get_tasks_acc()
print(f"a: {a}, b: {b}, c: {c}")

### Replay Strategy

In [ ]:
model4 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=7 ,include_top = True, num_classes = 10).to(device)

optimizer4 = Adam(model4.parameters(), lr=0.01, weight_decay=0)

storage_p = ReservoirSamplingBuffer(max_size=1500)

train_epochs = 4

replay = Replay(
    model=model4,
    optimizer=optimizer4,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    storage_policy = storage_p,
    device=device
)

replay.train(train_stream)
b,c = replay.test(test_stream)
a = replay.get_tasks_acc()
print(f"a: {a}, b: {b}, c: {c}")

### Latent Replay Strategy

In [ ]:
from avalanche.evaluation.metrics import ElapsedTime

model4 = PhiNet_v2(pretrained="./TestModel/7_Layers/Adam.pth", device = device, num_layers= 7, latent_layer_num=9).to(device)

optimizer4 = Adam(model4.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

t = ElapsedTime()

latent_replay = LatentReplay(
    model=model4,
    optimizer=optimizer4,
    criterion=criterion,
    train_mb_size=21,
    replay_mb_size = 107,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    rm_size = 1500,
    device=device
)

latent_replay.train(train_stream, test_stream, plotting = False)
t.update()
b,c = latent_replay.test(test_stream)
a = latent_replay.get_tasks_acc()
print(f"a: {a}, b: {b}, c: {c}")
t.update()
print(f"Inference time: {t.result():.2f} s")

mnist_tensor = torch.randn(1, 1, 28, 28)
mac = get_MAC(model4, mnist_tensor)